In [15]:
import pandas as pd
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import Descriptors
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn import neural_network as nn
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
import numpy as np
import matplotlib.pyplot as plt

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
df = pd.read_csv("Permuted Data")
df

,Standardized_SMILES,pchembl_value,MolWt,LogP,QED
0,NC(=O)c1c(-c2ccc(NC(=O)c3cccc(C(F)(F)F)c3)cc2)...,5.34,576.601,3.44580,0.304138
1,CCCCNC(=O)Nc1ccc(Oc2ncnc3cc(OC)c(OC)cc23)cc1,6.40,396.447,4.36090,0.549502
2,CC(=O)Nc1cn2nc(Oc3cccc(NC(=O)c4cccc(F)c4)c3)cc...,8.96,405.389,3.87140,0.525306
3,COc1cc(OC)cc(-c2cc3c[nH]c(=NCCCCO)cc3[nH]c2=NC...,8.10,467.570,3.26220,0.398199
4,Cc1cccc(NC(=O)Nc2ccc(-c3csc4[nH]cnc(=N)c34)cc2...,7.72,393.447,4.86239,0.399977
...,...,...,...,...,...
1636,Cc1[nH]nc2nccc(-c3ccc(NC(=O)Nc4cccnc4)cc3)c12,5.51,344.378,3.97232,0.523798
1637,O=C(Nc1ccc(F)cc1)C1(C(=O)Nc2ccc(Oc3cc[nH]c(=Nc...,6.59,501.493,5.07000,0.307360
1638,CCOC(=O)N=c1s[nH]c(-c2ccc(NC(=O)Nc3cc(C)ccc3F)...,7.30,457.487,3.99082,0.459966
1639,O=C(CSc1nc2ccccc2n2cnnc12)Nc1ccc(C(=O)N=c2ccc(...,8.41,489.948,3.73090,0.363128


In [17]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt


# Create a DeepChem dataset
tasks = ["pchembl_value"]
featurizer = dc.feat.CircularFingerprint(size=2048)
loader = dc.data.CSVLoader(tasks=tasks, feature_field="Standardized_SMILES", featurizer=featurizer)
dataset = loader.create_dataset("Permuted Data")

# Define the scaffold splitter
splitter = dc.splits.ScaffoldSplitter()

# Get scaffold split indices
train_valid_test = splitter.k_fold_split(dataset, k=10, seed=42)


In [18]:
#MLP

import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = nn.MLPRegressor(hidden_layer_sizes=(min(25,round(np.sqrt(X_train.shape[0])),)), random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1477, Test size: 164


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 1 Accuracy: (0.6214465446521465, 6.751708853835307e-19)
Fold 1 rmse: 1.1050102645938515
Fold 1 r_squared: 0.10755275706503375
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.6180519555201973
Fold 2 - Train size: 1477, Test size: 164


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 2 Accuracy: (0.47034575991043076, 2.0873128202787013e-10)
Fold 2 rmse: 0.8402922712585233
Fold 2 r_squared: -0.06276492943569534
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.6084286383546638
Fold 3 - Train size: 1477, Test size: 164


...

Fold 8 Accuracy: (0.6543046114514515, 2.0859201592246762e-21)
Fold 8 rmse: 0.8793940503592708
Fold 8 r_squared: 0.3184826901144747
Fold 8 Mean of Two Highest Tanimoto Similarities: 0.7190118279612702
Fold 9 - Train size: 1477, Test size: 164


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 9 Accuracy: (0.7597300051192853, 4.282782083542211e-32)
Fold 9 rmse: 0.8186925597075109
Fold 9 r_squared: 0.5504993352185943
Fold 9 Mean of Two Highest Tanimoto Similarities: 0.7303519830535362
Fold 10 - Train size: 1476, Test size: 165


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 10 Accuracy: (0.5993533417685363, 1.8174223398822987e-17)
Fold 10 rmse: 1.0490853322344236
Fold 10 r_squared: 0.03987760559404474
Fold 10 Mean of Two Highest Tanimoto Similarities: 0.7062209056733456
Mean Pearson R: 0.32835041913993035
Overall Mean of Two Highest Tanimoto Similarities: 0.6838428267707414


In [19]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("scaffold_cv_results_MLP_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,C=S(=N)(O)c1ccc(N=c2[nH]cc(Br)c(=NC(C)CO)[nH]2...,7.40,7.363969
1,1,CC(N=c1[nH]c(=Nc2ccc(S(=O)(CC=O)=NS(=O)(=O)CC[...,6.80,6.766265
2,1,CCOC(=O)N=S(=O)(CC)c1ccc(N=c2[nH]cc(Br)c(=NC(C...,7.04,7.239587
3,1,COc1cc(N=c2[nH]cc(Br)c(=NC(C)C(C)O)[nH]2)ccc1S...,7.16,7.221433
4,1,CCOC(=O)N=S(C)(=O)c1ccc(N=c2[nH]cc(Br)c(=NC(C)...,7.23,7.224180
...,...,...,...,...
14764,10,CN(c1ccc(NS(=O)(=O)Cc2ccccc2)cc1)c1cc[nH]c(=Nc...,6.57,6.529790
14765,10,Cc1cc2c(F)c(Oc3[nH]cnc(=N)c3C=NOCC3CCCCC3)ccc2...,6.38,6.359548
14766,10,COc1ccc(N(C(=O)Oc2c(C)cccc2C)c2cc[nH]c(=Nc3ccc...,6.21,6.463578
14767,10,O=C(Nc1ccc(Oc2ccccc2)cc1)c1scnc1CCc1ccncc1,5.72,6.173121


In [20]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("scaffold_cv_results_MLP_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,CCN(CC)CCOc1cc2[nH]cnc(=Nc3ccc(F)c(Cl)c3)c2cc1OC,5.16,4.689494
1,1,COc1cc2[nH]cnc(=Nc3ccc(Br)cc3)c2cc1OCCCCCC(O)=NO,6.82,4.497726
2,1,COc1cc2[nH]cnc(=Nc3ccccc3F)c2cc1OCCCCC(O)=NO,6.08,5.118017
3,1,COc1cc2[nH]cnc(=Nc3ccccc3F)c2cc1OCCCC(O)=NO,6.06,5.204487
4,1,CCCOc1cc2[nH]cnc(=Nc3ccc(NC(=O)OCC)c(Cl)c3)c2c...,6.30,5.974985
...,...,...,...,...
1636,10,Cc1cccc(NC(=O)N(Cc2ccc(-c3[nH]nc4c3Cc3cc(CN5CC...,7.68,6.243485
1637,10,O=C(Nc1ccc(N2CCn3ccc4ncnc2c43)cc1)Nc1cccc(C(F)...,8.15,6.478442
1638,10,c1cc(N=c2[nH]cnc3cc(OCCN4CCCC4)cc(OC4CCOCC4)c2...,7.30,6.804713
1639,10,Cc1cccc(NC(=O)Nc2ccc(-c3ccc(OCCN4CCCC4=O)c4[nH...,7.60,7.832419


In [21]:
#RF

In [22]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = RandomForestRegressor(n_estimators=min(25,round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1477, Test size: 164
Fold 1 Accuracy: (0.7257402592252981, 4.164085410113753e-28)
Fold 1 rmse: 0.8489514047270759
Fold 1 r_squared: 0.473236602340263
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.6180519555201973
Fold 2 - Train size: 1477, Test size: 164
Fold 2 Accuracy: (0.5898618744224825, 9.514463717839633e-17)
Fold 2 rmse: 0.6583073626039255
Fold 2 r_squared: 0.3477202424871312
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.6084286383546638
Fold 3 - Train size: 1477, Test size: 164
Fold 3 Accuracy: (0.7575112219272943, 8.164844045270744e-32)
Fold 3 rmse: 0.6859652654025229
Fold 3 r_squared: 0.5045381370555528
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.7270965615406307
Fold 4 - Train size: 1477, Test size: 164
Fold 4 Accuracy: (0.4885407264481295, 3.212549588239616e-11)
Fold 4 rmse: 0.9195933835987166
Fold 4 r_squared: -0.046592280762461025
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.6573044855972131
Fold 5 - Train size: 1477, Test si

In [23]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("scaffold_cv_results_RF_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,C=S(=N)(O)c1ccc(N=c2[nH]cc(Br)c(=NC(C)CO)[nH]2...,7.40,7.1400
1,1,CC(N=c1[nH]c(=Nc2ccc(S(=O)(CC=O)=NS(=O)(=O)CC[...,6.80,6.7758
2,1,CCOC(=O)N=S(=O)(CC)c1ccc(N=c2[nH]cc(Br)c(=NC(C...,7.04,7.0488
3,1,COc1cc(N=c2[nH]cc(Br)c(=NC(C)C(C)O)[nH]2)ccc1S...,7.16,7.1736
4,1,CCOC(=O)N=S(C)(=O)c1ccc(N=c2[nH]cc(Br)c(=NC(C)...,7.23,7.2636
...,...,...,...,...
14764,10,CN(c1ccc(NS(=O)(=O)Cc2ccccc2)cc1)c1cc[nH]c(=Nc...,6.57,6.7504
14765,10,Cc1cc2c(F)c(Oc3[nH]cnc(=N)c3C=NOCC3CCCCC3)ccc2...,6.38,6.6032
14766,10,COc1ccc(N(C(=O)Oc2c(C)cccc2C)c2cc[nH]c(=Nc3ccc...,6.21,6.3504
14767,10,O=C(Nc1ccc(Oc2ccccc2)cc1)c1scnc1CCc1ccncc1,5.72,6.0568


In [24]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("scaffold_cv_results_RF_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,CCN(CC)CCOc1cc2[nH]cnc(=Nc3ccc(F)c(Cl)c3)c2cc1OC,5.16,5.8312
1,1,COc1cc2[nH]cnc(=Nc3ccc(Br)cc3)c2cc1OCCCCCC(O)=NO,6.82,6.2836
2,1,COc1cc2[nH]cnc(=Nc3ccccc3F)c2cc1OCCCCC(O)=NO,6.08,6.4280
3,1,COc1cc2[nH]cnc(=Nc3ccccc3F)c2cc1OCCCC(O)=NO,6.06,6.4320
4,1,CCCOc1cc2[nH]cnc(=Nc3ccc(NC(=O)OCC)c(Cl)c3)c2c...,6.30,6.3608
...,...,...,...,...
1636,10,Cc1cccc(NC(=O)N(Cc2ccc(-c3[nH]nc4c3Cc3cc(CN5CC...,7.68,6.8160
1637,10,O=C(Nc1ccc(N2CCn3ccc4ncnc2c43)cc1)Nc1cccc(C(F)...,8.15,7.3284
1638,10,c1cc(N=c2[nH]cnc3cc(OCCN4CCCC4)cc(OC4CCOCC4)c2...,7.30,6.4388
1639,10,Cc1cccc(NC(=O)Nc2ccc(-c3ccc(OCCN4CCCC4=O)c4[nH...,7.60,7.4362


In [25]:
#xgboost

In [26]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = xgb.XGBRegressor(n_estimators=min(25, round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1477, Test size: 164
Fold 1 Accuracy: (0.6942128107990445, 6.570292722524195e-25)
Fold 1 rmse: 0.8699882965538878
Fold 1 r_squared: 0.4468069078022139
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.6180519555201973
Fold 2 - Train size: 1477, Test size: 164
Fold 2 Accuracy: (0.5346530288891632, 1.6550488260335224e-13)
Fold 2 rmse: 0.6957775996386182
Fold 2 r_squared: 0.2713526970614686
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.6084286383546638
Fold 3 - Train size: 1477, Test size: 164
Fold 3 Accuracy: (0.7685539842080461, 3.0657167951591517e-33)
Fold 3 rmse: 0.6514699728154998
Fold 3 r_squared: 0.5531160190200528
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.7270965615406307
Fold 4 - Train size: 1477, Test size: 164
Fold 4 Accuracy: (0.2879202267003997, 0.000185154529035226)
Fold 4 rmse: 1.0093891685916223
Fold 4 r_squared: -0.2609652889739591
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.6573044855972131
Fold 5 - Train size: 1477, Test si

In [27]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("scaffold_cv_results_xgboost_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,C=S(=N)(O)c1ccc(N=c2[nH]cc(Br)c(=NC(C)CO)[nH]2...,7.40,7.046021
1,1,CC(N=c1[nH]c(=Nc2ccc(S(=O)(CC=O)=NS(=O)(=O)CC[...,6.80,6.727025
2,1,CCOC(=O)N=S(=O)(CC)c1ccc(N=c2[nH]cc(Br)c(=NC(C...,7.04,6.831810
3,1,COc1cc(N=c2[nH]cc(Br)c(=NC(C)C(C)O)[nH]2)ccc1S...,7.16,6.954276
4,1,CCOC(=O)N=S(C)(=O)c1ccc(N=c2[nH]cc(Br)c(=NC(C)...,7.23,7.143680
...,...,...,...,...
14764,10,CN(c1ccc(NS(=O)(=O)Cc2ccccc2)cc1)c1cc[nH]c(=Nc...,6.57,7.014631
14765,10,Cc1cc2c(F)c(Oc3[nH]cnc(=N)c3C=NOCC3CCCCC3)ccc2...,6.38,6.864052
14766,10,COc1ccc(N(C(=O)Oc2c(C)cccc2C)c2cc[nH]c(=Nc3ccc...,6.21,6.560207
14767,10,O=C(Nc1ccc(Oc2ccccc2)cc1)c1scnc1CCc1ccncc1,5.72,6.384591


In [28]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("scaffold_cv_results_xgboost_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,CCN(CC)CCOc1cc2[nH]cnc(=Nc3ccc(F)c(Cl)c3)c2cc1OC,5.16,5.903025
1,1,COc1cc2[nH]cnc(=Nc3ccc(Br)cc3)c2cc1OCCCCCC(O)=NO,6.82,6.038775
2,1,COc1cc2[nH]cnc(=Nc3ccccc3F)c2cc1OCCCCC(O)=NO,6.08,5.931204
3,1,COc1cc2[nH]cnc(=Nc3ccccc3F)c2cc1OCCCC(O)=NO,6.06,5.931204
4,1,CCCOc1cc2[nH]cnc(=Nc3ccc(NC(=O)OCC)c(Cl)c3)c2c...,6.30,5.936219
...,...,...,...,...
1636,10,Cc1cccc(NC(=O)N(Cc2ccc(-c3[nH]nc4c3Cc3cc(CN5CC...,7.68,6.882324
1637,10,O=C(Nc1ccc(N2CCn3ccc4ncnc2c43)cc1)Nc1cccc(C(F)...,8.15,7.194548
1638,10,c1cc(N=c2[nH]cnc3cc(OCCN4CCCC4)cc(OC4CCOCC4)c2...,7.30,6.296010
1639,10,Cc1cccc(NC(=O)Nc2ccc(-c3ccc(OCCN4CCCC4=O)c4[nH...,7.60,7.608147
